<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-4/2-similarity-search-level-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-4/2-similarity-search-level-3.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This code is part of [Chapter 4 - Building a Reverse Image Search Engine: Understanding Embeddings](https://learning.oreilly.com/library/view/practical-deep-learning/9781492034858/ch04.html).

Note: In order to run this notebook on Google Colab you need to [follow these instructions](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb#scrollTo=WzIRIt9d2huC) so that the local data such as the images are available in your Google Drive.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive

    drive.mount("/content/gdrive")

    IS_COLAB_ENV = True
except:
    IS_COLAB_ENV = False
IS_COLAB_ENV

Mounted at /content/gdrive


True

# Similarity Search

## Level 3

So far we experimented with different visualization techniques on the results, t-SNE and PCA on the results. Now we will calculate the accuracies of the features obtained from the pretrained and finetuned models. The finetuning here follows the same finetuning technique we learnt in Chapter 2.

In [2]:
if IS_COLAB_ENV:
    !mkdir -p ../../datasets
    !pip install gdown
    !gdown https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp --output ../../datasets/caltech101.tar.gz
    !tar -xvzf ../../datasets/caltech101.tar.gz --directory ../../datasets
    !mv ../../datasets/101_ObjectCategories ../../datasets/caltech101
    !rm -rf ../../datasets/caltech101/BACKGROUND_Google

Streaming output truncated to the last 5000 lines.
101_ObjectCategories/chair/image_0005.jpg
101_ObjectCategories/chair/image_0006.jpg
101_ObjectCategories/chair/image_0007.jpg
101_ObjectCategories/chair/image_0008.jpg
101_ObjectCategories/chair/image_0010.jpg
101_ObjectCategories/chair/image_0011.jpg
101_ObjectCategories/chair/image_0012.jpg
101_ObjectCategories/chair/image_0013.jpg
101_ObjectCategories/chair/image_0014.jpg
101_ObjectCategories/chair/image_0016.jpg
101_ObjectCategories/chair/image_0017.jpg
101_ObjectCategories/chair/image_0018.jpg
101_ObjectCategories/chair/image_0019.jpg
101_ObjectCategories/chair/image_0020.jpg
101_ObjectCategories/chair/image_0022.jpg
101_ObjectCategories/chair/image_0023.jpg
101_ObjectCategories/chair/image_0024.jpg
101_ObjectCategories/chair/image_0025.jpg
101_ObjectCategories/chair/image_0026.jpg
101_ObjectCategories/chair/image_0028.jpg
101_ObjectCategories/chair/image_0029.jpg
101_ObjectCategories/chair/image_0030.jpg
101_ObjectCategories/chai

In [3]:
import numpy as np
import pickle
from tqdm.notebook import tqdm as tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors

import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

For these experiments we will use the same features of the Caltech101 dataset that we were using before.

Let's utilize the features from the previously trained model.

In [4]:
model_architecture = "resnet"
model_features_path = f"data/features-caltech101-{model_architecture}.pickle"
generator_classes_path = "data/class_ids-caltech101.pickle"
filenames_path = "data/filenames-caltech101.pickle"

if IS_COLAB_ENV:
    generator_classes_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{generator_classes_path}"
    filenames_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filenames_path}"
    model_features_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{model_features_path}"

In [5]:
filenames = pickle.load(open(filenames_path, "rb"))
feature_list = pickle.load(open(model_features_path, "rb"))
class_ids = pickle.load(open(generator_classes_path, "rb"))

num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  8677
Number of features per image =  2048


First, let's make a helper function that calculates the accuracy of the resultant features using the nearest neighbors brute force algorithm.

In [6]:
def calculate_accuracy(feature_list, num_nearest_neighbors=5):
    neighbors = NearestNeighbors(
        n_neighbors=num_nearest_neighbors, algorithm="brute", metric="euclidean"
    ).fit(feature_list)
    distances, np_indices = neighbors.kneighbors(feature_list)

    neighbors_class_ids = class_ids[np_indices]
    class_equalities = np.equal(
        neighbors_class_ids[:, [0]], neighbors_class_ids[:, 1:num_nearest_neighbors]
    )

    print(f"Accuracy is: {round(class_equalities.mean() * 100.0, 2)}")

### 1. Accuracy of Brute Force over Caltech101 features

In [7]:
# Calculate accuracy
calculate_accuracy(feature_list)

Accuracy is: 88.46


### 2. Accuracy of Brute Force over the PCA compressed Caltech101 features

In [8]:
num_feature_dimensions = 100
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list[:])

Let's calculate accuracy over the compressed features.

In [9]:
calculate_accuracy(feature_list_compressed)

Accuracy is: 88.52


### 3. Accuracy of Brute Force over the finetuned Caltech101 features

In [10]:
model_architecture = "resnet-finetuned"
model_features_path = f"data/features-caltech101-{model_architecture}.pickle"
generator_classes_path = "data/class_ids-caltech101.pickle"
filenames_path = "data/filenames-caltech101.pickle"

if IS_COLAB_ENV:
    generator_classes_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{generator_classes_path}"
    filenames_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filenames_path}"
    model_features_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{model_features_path}"

In [11]:
filenames = pickle.load(open(filenames_path, "rb"))
feature_list = pickle.load(open(model_features_path, "rb"))
class_ids = pickle.load(open(generator_classes_path, "rb"))

num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  8677
Number of features per image =  101


In [12]:
# Calculate accuracy
calculate_accuracy(feature_list)

Accuracy is: 88.37


### 4. Accuracy of Brute Force over the PCA compressed finetuned Caltech101 features

In [13]:
# Perform PCA
num_feature_dimensions = 100
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list[:])

In [14]:
# Calculate accuracy over the compressed features
calculate_accuracy(feature_list_compressed[:])

Accuracy is: 88.33


### Accuracy

These results lead to the accuracy on Caltech101. Repeating Level 3 on the Caltech256 features we get its corresponding accuracy.

Accuracy on Caltech101.

| Algorithm | Accuracy using Pretrained features| Accuracy using Finetuned features |
|-------------|----------------------------|------------------------|
| Brute Force | 87.06 | 89.48 |
| PCA + Brute Force | 87.65  |  89.39 |


Accuracy on Caltech256.

| Algorithm | Accuracy using Pretrained features| Accuracy using Finetuned features |
|-------------|----------------------------|------------------------|
| Brute Force | 58.38 | 96.01 |
| PCA + Brute Force | 56.64  | 95.34|